## KBReport에서 타자 스탯 수집하기

In [ ]:
# 필요한 패키지 호출
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import time

In [ ]:
# 요청 URL 설정
url = 'http://www.kbreport.com/leader/list/ajax'

In [ ]:
# Body 파라미터 설정
data = {'rows': 20,
        'order': 'oWAR',
        'orderType': 'DESC',
        'year_from': 2020,
        'year_to': 2020,
        'gameType': 'R', 
        'page': 1}

In [ ]:
# HTTP Request 
res = requests.post(url = url, data = data)

In [ ]:
# 응답 상태코드 확인
res.status_code

In [ ]:
# 요청 헤더 확인
res.headers

In [ ]:
# 응답 콘텐트 확인 (bytes 객체로 출력됨)
res.content

In [ ]:
# UTF-8으로 디코딩 (한글이 제대로 보이도록)
res.content.decode('UTF-8')

In [ ]:
# BeautifulSoup을 이용하여 HTML parsing
html = bts(res.content, 'html.parser')
html

In [ ]:
# <table> 태그만 선택 
tbl = html.select('table.ltb-table')
tbl

In [ ]:
# 문자열을 데이터프레임으로 읽어오기
df = pd.read_html(str(tbl))[0]
df

In [ ]:
# 불필요한 컬럼 삭제 
df.drop('#', axis = 1, inplace = True)
df

## 반복문 실행

In [ ]:
# 최종 데이터를 저장할 빈 데이터프레임 생성
result = pd.DataFrame()

# 수집할 연도 설정
year = 2020

# 첫 번째 페이지 설정
page = 1

In [ ]:
# 반복문 실행
while page:
    
    # 출력 현황 확인 
    print('현재', page, '페이지 작업 중!')
    
    # 요청 URL 설정
    url = 'http://www.kbreport.com/leader/list/ajax'

    # Body 파라미터 설정
    data = {'rows': 20,
            'order': 'oWAR',
            'orderType': 'DESC',
            'year_from': year,
            'year_to': year,
            'gameType': 'R', 
            'page': page}
    
    # HTTP Request 
    res = requests.post(url = url, data = data)
    
    # BeautifulSoup을 이용하여 HTML parsing
    html = bts(res.content, 'html.parser')
    
    # <table> 태그만 선택 
    tbl = html.select('table.ltb-table')
    
    # 문자열을 pd table 형태로 변환
    df = pd.read_html(str(tbl))[0]
    
    # 불필요한 컬럼 삭제 
    df.drop('#', axis = 1, inplace = True)
    
    # result에 추가 
    result = result.append(df, ignore_index = True)
    
    # df의 행 길이가 20 미만이면 중단 
    if df.shape[0] < 20: 
        print('---> 작업을 종료합니다.')
        break
    
    # 다음 페이지 설정 
    page += 1
    
    # 1초간 멈춤 
    time.sleep(1)

In [ ]:
# 작업 결과 확인
result.info()

In [ ]:
# 상위 20명만 출력
result.head(20)

## End of Document